In [6]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [4]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name = "rag", dimension = 1536, metric = "cosine", spec=ServerlessSpec(cloud = "aws", region = "us-east-1")
)

In [5]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Carter',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Excellent professor! She explains concepts very clearly and is always available for help.'},
 {'professor': 'Dr. John Smith',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Great lectures, but sometimes moves too quickly through material.'},
 {'professor': 'Dr. Anna Lee',
  'subject': 'Chemistry',
  'stars': 3,
  'review': 'Average experience. The lectures were fine, but the exams were much harder than expected.'},
 {'professor': 'Dr. Robert Brown',
  'subject': 'History',
  'stars': 2,
  'review': 'Not very engaging. The lectures were dry and difficult to follow.'},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Amazing professor! She made the subject interesting and was always supportive.'},
 {'professor': 'Dr. Michael Davis',
  'subject': 'Economics',
  'stars': 4,
  'review': 'Good professor, but assignments were a bit too challenging.'},
 {'professo

In [12]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model = "text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
    "values": embedding,
    "id": review["professor"],
    "metadata": {
        "review": review["review"],
        "subject": review["subject"],
        "stars": review["stars"]
    }
    })

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [10]:
processed_data[0]

IndexError: list index out of range

In [13]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 22 Aug 2024 00:53:57 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '44', 'x-pinecone-request-id': '8650112721146788023', 'x-envoy-upstream-service-time': '45', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}
